<a href="https://colab.research.google.com/github/nagen01/Colab_Projects/blob/master/ham_spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
df = pd.read_csv('/content/smsspamcollection.tsv',sep='\t')

In [0]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [0]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [0]:


from sklearn.model_selection import train_test_split

In [0]:
X = df['message']
y = df['label']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=42)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer 

In [0]:
count_vect = CountVectorizer()

In [0]:
#Fitting the data : Creating vocab, counting the unique words  etc....... 
# count_vect.fit(X_train)
#Transforming it : crating sparse matrix out of the unique words
#X_train_counts = count_vect.transform(X_train)

#Both can be done in fit_transfrm
X_train_counts = count_vect.fit_transform(X_train)

In [0]:
X_train_counts

<3733x7082 sparse matrix of type '<class 'numpy.int64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [0]:
X_train_counts.shape

(3733, 7082)

In [0]:
X_train.shape

(3733,)

In [0]:
#TFIDF 
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_tf = TfidfTransformer()

In [0]:
X_train_tfidf_tf = tfidf_tf.fit_transform(X_train_counts)

In [0]:
X_train_tfidf_tf.shape

(3733, 7082)

In [0]:
#CountVectorizer and TfiffTransformation can be combined in one step with TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer()

In [0]:
X_train_tfidf_vect = tfidf_vect.fit_transform(X_train)

In [0]:
X_train_tfidf_vect.shape

(3733, 7082)

In [0]:
from sklearn.svm import LinearSVC
clf = LinearSVC()

In [0]:
model = clf.fit(X_train_tfidf_vect,y_train)

In [0]:
prediction = model.predict(X_test)

ValueError: ignored

In [0]:
#Here only training data is converted in sparse matrix, if we need to conver test data as well, we will have to follow all the process
#but we have "pipeline" feature, to simplify this kind of work......

from sklearn.pipeline import Pipeline
text_clf = Pipeline([('tfidf_vect',TfidfVectorizer()),('clf',LinearSVC())])

In [0]:
text_clf.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('tfidf_vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
          

In [0]:
pred = text_clf.predict(X_test)

In [0]:
from sklearn import metrics

In [0]:
print(metrics.confusion_matrix(y_test,pred))

[[1586    7]
 [  12  234]]


In [0]:
text_clf.predict(['how are you doing Today?'])

array(['ham'], dtype=object)